# 集成了水印、美学、CLIP模型，用于给图文质量打分

In [ ]:
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import torch
import timm
from torchvision import transforms as T
import open_clip
import torch
from transformers import BertModel, BertTokenizer
from PIL import Image

class AestheticsMLP(pl.LightningModule):
    # 美学判别器是基于CLIP的基础上接了一个MLP
    def __init__(self, input_size, xcol='emb', ycol='avg_rating'):
        super().__init__()
        self.input_size = input_size
        self.xcol = xcol
        self.ycol = ycol
        self.layers = nn.Sequential(
            nn.Linear(self.input_size, 1024),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 128),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            #nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(64, 16),
            #nn.ReLU(),

            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
            x = batch[self.xcol]
            y = batch[self.ycol].reshape(-1, 1)
            x_hat = self.layers(x)
            loss = F.mse_loss(x_hat, y)
            return loss
    
    def validation_step(self, batch, batch_idx):
        x = batch[self.xcol]
        y = batch[self.ycol].reshape(-1, 1)
        x_hat = self.layers(x)
        loss = F.mse_loss(x_hat, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


class WaterMarkModel(nn.Module):
    def __init__(self, model_path='./watermark_model_v1.pt'):
        super(WaterMarkModel, self).__init__()
        # model definition
        self.model = timm.create_model(
                'efficientnet_b3a', pretrained=True, num_classes=2)

        self.model.classifier = nn.Sequential(
            # 1536 is the orginal in_features
            nn.Linear(in_features=1536, out_features=625),
            nn.ReLU(),  # ReLu to be the activation function
            nn.Dropout(p=0.3),
            nn.Linear(in_features=625, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=2),
        )
        self.model.load_state_dict(torch.load(model_path))
    def forward(self, x):
        return self.model(x)

In [ ]:
class FilterSystem:
    def __init__(
                    self, 
                    clip_model_path="IDEA-CCNL/Taiyi-CLIP-RoBERTa-102M-ViT-L-Chinese",
                    aesthetics_model_path="./ava+logos-l14-linearMSE.pth",
                    watermark_model_path="./watermark_model_v1.pt"
                ):
        self.clip_model_path = clip_model_path
        self.aesthetics_model_path = aesthetics_model_path
        self.watermark_model_path = watermark_model_path

    def init_clip_model(self, ):
        # 此处初始化clip模型，返回模型、tokenizer、processor
        text_encoder = BertModel.from_pretrained(self.clip_model_path).eval().cuda()
        text_tokenizer = BertTokenizer.from_pretrained(self.clip_model_path)
        clip_model, _, processor = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')
        clip_model = clip_model.eval().cuda()
        self.text_encoder, self.text_tokenizer, self.clip_model, self.processor = text_encoder, text_tokenizer, clip_model, processor
        print("clip model loaded")
        return None

    def init_aesthetics_model(self, ):
        # 此处初始化美学模型
        self.aesthetics_model = AestheticsMLP(768)
        self.aesthetics_model.load_state_dict(torch.load(self.aesthetics_model_path))
        self.aesthetics_model.eval().cuda()
        print("aesthetics model loaded")
        return None

    def init_watermark_model(self, ):
        self.watermark_model = WaterMarkModel(self.watermark_model_path)
        self.watermark_model.eval().cuda()
        self.watermark_processor =  T.Compose([
                                                T.Resize((256, 256)),
                                                T.ToTensor(),
                                                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                            ])
        print("watermark model loaded")
        return None

    def get_image_feature(self, images):
        # 此处返回图像的特征向量
        if isinstance(images, list):
            images = torch.stack([self.processor(image) for image in images]).cuda()
        elif isinstance(images, torch.Tensor):
            images = images.cuda()

        with torch.no_grad():
            image_features = self.clip_model.encode_image(images)
            image_features /= image_features.norm(dim=1, keepdim=True)
        return image_features
    
    def get_text_feature(self, text):
        # 此处返回文本的特征向量
        if isinstance(text, list) or isinstance(text, str):
            text = self.text_tokenizer(text, return_tensors='pt', padding=True)['input_ids'].cuda()
        elif isinstance(text, torch.Tensor):
            text = text.cuda()

        with torch.no_grad():
            text_features = self.text_encoder(text)[1]
            text_features /= text_features.norm(dim=1, keepdim=True)
        return text_features

    def calculate_clip_score(self, features1, features2):
        # 此处2个特征向量的相似度，输入可以是 图片+文本、文本+文本、图片+图片。
        # 返回的是相似度矩阵，维度为 f1.shape[0] * f2.shape[0]
        score_matrix =  features1 @ features2.t()
        return score_matrix

    def get_aesthetics_score(self, features):
        # 此处返回美学分数，传入的是CLIP的feature, 先计算get_image_feature在传入此函数~(模型是ViT-L-14)
        with torch.no_grad():
            scores = self.aesthetics_model(features)
            scores = scores[:, 0].detach().cpu().numpy()
        return scores
    
    def get_watermark_score(self, images):
        if isinstance(images, list):
            images = torch.stack([self.watermark_processor(image) for image in images]).cuda()
        elif isinstance(images, torch.Tensor):
            images = images.cuda()
        with torch.no_grad():
            pred = self.watermark_model(images)
            watermark_scores = F.softmax(pred, dim=1)[:,0].detach().cpu().numpy()

        return watermark_scores

## 小规模数据测试

In [ ]:
demo = FilterSystem()
demo.init_clip_model()
demo.init_aesthetics_model()
demo.init_watermark_model()

In [ ]:
image_path = './demo_images/watermark_example.png'
image_path2 = './demo_images/mengna.jpg'
image_path3 = './demo_images/shuiyin.jpg'
image_path4 = './demo_images/1.jpg'
image_demo =  [Image.open(image_path).convert('RGB'), Image.open(image_path2).convert('RGB'), Image.open(image_path3).convert('RGB'), Image.open(image_path4).convert('RGB')]
image_feature = demo.get_image_feature(image_demo,)  # 计算图片特征，传入图片列表，一般而言，可以在数据库保存这个东西，用于响应文本query
aes_score = demo.get_aesthetics_score(image_feature)  # 计算美学分数，传入图片特征，一般而言，可以在数据库保存这个东西，用于响应文本query
print(aes_score)

In [ ]:
text_demo = ['一副很美的画','港口小船', '蒙娜丽莎'] # 这里也可以只有一个文本，也就是query
text_feature = demo.get_text_feature(text_demo) # 计算文本特征，传入文本列表
similarity = demo.calculate_clip_score(image_feature, text_feature)  # 计算相似度
print(similarity)

In [ ]:
watermark_score = demo.get_watermark_score(image_demo)
print(watermark_score)

## 读取处理保存（单个进程）

In [ ]:
# data setting
root_path = "./project/dataset/laion_chinese_cwf/image_part00"
all_folders = sorted(os.listdir(root_path))

In [ ]:
# model setting
filter_model = FilterSystem()
filter_model.init_clip_model()
filter_model.init_aesthetics_model()
filter_model.init_watermark_model()

In [ ]:
from model import FilterSystem
from dataset import TxtDataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd

def sub_process(filter_model, each_folder_path):
    each_dataset = TxtDataset(each_folder_path)
    each_dataloader = DataLoader(each_dataset, batch_size=8, shuffle=False, num_workers=8)

    image_paths = []
    aes_scores = []
    clip_scores = []
    watermark_scores = []
    for iii, (batch_image_paths, texts,) in enumerate(tqdm(each_dataloader)):
        images =  [Image.open(each_image_path).convert("RGB") for each_image_path in batch_image_paths]
        image_paths.extend(batch_image_paths)

        image_features = filter_model.get_image_feature(images,)  # 计算图片特征，传入图片列表，一般而言，可以在数据库保存这个东西，用于响应文本query
        aes_score = filter_model.get_aesthetics_score(image_features)  # 计算美学分数，传入图片特征，一般而言，可以在数据库保存这个东西，用于响应文本query
        aes_scores.extend(aes_score)

        text_features = filter_model.get_text_feature(list(texts)) # 计算文本特征，传入文本列表
        clip_score = filter_model.calculate_clip_score(image_features, text_features)  # 计算相似度
        clip_scores.extend(torch.diagonal(clip_score).detach().cpu().numpy())  # 需要取对角线，只需要自己和对应文本的相似度

        watermark_score = filter_model.get_watermark_score(images)  # 计算水印分数，传入图片路径列表
        watermark_scores.extend(watermark_score)
        
        # print('aes_score:', aes_score, '\n',
        #     'clip_score:', clip_score, '\n',
        #     'watermark_score:', watermark_score, '\n',
        #     'image_paths:', image_paths, '\n',
        #     'texts:', texts)
        
    score_pd = pd.DataFrame({'image_path': image_paths, 'aes_score': aes_scores, 'clip_score': clip_scores, 'watermark_score': watermark_scores})
    score_pd.to_csv(os.path.join(each_folder_path, 'score.csv'), index=False)
    print('save score.csv in {}'.format(each_folder_path), '\n', '-'*20)

for each_folder in all_folders[:10]:
    each_folder_path = os.path.join(root_path, each_folder)
    sub_process(filter_model, each_folder_path)

In [ ]:
from model import FilterSystem
from dataset import TxtDataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, wait, ALL_COMPLETED

p = ProcessPoolExecutor(max_workers=4)

def sub_process(filter_model, each_folder_path):
    each_dataset = TxtDataset(each_folder_path)
    each_dataloader = DataLoader(each_dataset, batch_size=8, shuffle=False, num_workers=8)

    image_paths = []
    aes_scores = []
    clip_scores = []
    watermark_scores = []
    for iii, (batch_image_paths, texts,) in enumerate(each_dataloader):
        images =  [Image.open(each_image_path) for each_image_path in batch_image_paths]
        image_paths.extend(batch_image_paths)

        image_features = filter_model.get_image_feature(images,)  # 计算图片特征，传入图片列表，一般而言，可以在数据库保存这个东西，用于响应文本query
        aes_score = filter_model.get_aesthetics_score(image_features)  # 计算美学分数，传入图片特征，一般而言，可以在数据库保存这个东西，用于响应文本query
        aes_scores.extend(aes_score)

        text_features = filter_model.get_text_feature(list(texts)) # 计算文本特征，传入文本列表
        clip_score = filter_model.calculate_clip_score(image_features, text_features)  # 计算相似度
        clip_scores.extend(torch.diagonal(clip_score).detach().cpu().numpy())  # 需要取对角线，只需要自己和对应文本的相似度

        watermark_score = filter_model.get_watermark_score(images)  # 计算水印分数，传入图片路径列表
        watermark_scores.extend(watermark_score)
        
        # print('aes_score:', aes_score, '\n',
        #     'clip_score:', clip_score, '\n',
        #     'watermark_score:', watermark_score, '\n',
        #     'image_paths:', image_paths, '\n',
        #     'texts:', texts)
        
    score_pd = pd.DataFrame({'image_path': image_paths, 'aes_score': aes_scores, 'clip_score': clip_scores, 'watermark_score': watermark_scores})
    score_pd.to_csv(os.path.join(each_folder_path, 'score.csv'), index=False)
    print('save score.csv in {}'.format(each_folder_path), '\n', '-'*20)

for each_folder in all_folders[:10]:
    each_folder_path = os.path.join(root_path, each_folder)
    f1 = p.submit(sub_process, model_pool[0], each_folder_path)
    f2 = p.submit(sub_process, model_pool[1], each_folder_path)
    f3 = p.submit(sub_process, model_pool[2], each_folder_path)
    f4 = p.submit(sub_process, model_pool[3], each_folder_path)
    res = wait([f1, f2, f3, f4], return_when=ALL_COMPLETED)
p.shutdown()


In [ ]:
# 用model pool来开启4进程跑
model_pool = [FilterSystem() for i in range(4)]
for model in model_pool:
    model.init_clip_model()
    model.init_aesthetics_model()
    model.init_watermark_model()

In [ ]:
print(aes_scores, clip_scores, watermark_scores)

In [ ]:
print('image_paths:', image_paths, '\n',  'texts:', texts)

# pytorch lightning + multi process.

In [1]:
import pytorch_lightning as pl

class ScoreSystem(pl.LightningModule):
    def __init__(Self):
        super().__init__()
        self.text_encoder, self.text_tokenizer, self.clip_model, self.processor = self.init_clip_model()
        self.aesthetics_model = self.init_aesthetics_model()
        self.watermark_model, self.watermark_processor = self.init_watermark_model()

    def init_clip_model(self):
        text_encoder = BertModel.from_pretrained(self.clip_model_path).eval().cuda()
        text_tokenizer = BertTokenizer.from_pretrained(self.clip_model_path)
        clip_model, _, processor = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')
        clip_model = clip_model.eval().cuda()
        print("clip model loaded")
        return text_encoder, text_tokenizer, clip_model, processor

    def init_aesthetics_model(self, ):
        # 此处初始化美学模型
        aesthetics_model = AestheticsMLP(768)
        aesthetics_model.load_state_dict(torch.load(self.aesthetics_model_path)).eval().cuda()
        print("aesthetics model loaded")
        return aesthetics_model

    def init_watermark_model(self, ):
        watermark_model = WaterMarkModel(self.watermark_model_path)
        watermark_model.eval().cuda()
        watermark_processor =  T.Compose([
                                                T.Resize((256, 256)),
                                                T.ToTensor(),
                                                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                            ])
        print("watermark model loaded")
        return watermark_model, watermark_processor

    def get_image_feature(self, images):
        # 此处返回图像的特征向量
        if isinstance(images, list):
            images = torch.stack([self.processor(image) for image in images]).cuda()
        elif isinstance(images, torch.Tensor):
            images = images.cuda()

        with torch.no_grad():
            image_features = self.clip_model.encode_image(images)
            image_features /= image_features.norm(dim=1, keepdim=True)
        return image_features
    
    def get_text_feature(self, text):
        # 此处返回文本的特征向量
        if isinstance(text, list) or isinstance(text, str):
            text = self.text_tokenizer(text, return_tensors='pt', padding=True)['input_ids'].cuda()
        elif isinstance(text, torch.Tensor):
            text = text.cuda()

        with torch.no_grad():
            text_features = self.text_encoder(text)[1]
            text_features /= text_features.norm(dim=1, keepdim=True)
        return text_features

    def calculate_clip_score(self, features1, features2):
        # 此处2个特征向量的相似度，输入可以是 图片+文本、文本+文本、图片+图片。
        # 返回的是相似度矩阵，维度为 f1.shape[0] * f2.shape[0]
        score_matrix =  features1 @ features2.t()
        return score_matrix

    def get_aesthetics_score(self, features):
        # 此处返回美学分数，传入的是CLIP的feature, 先计算get_image_feature在传入此函数~(模型是ViT-L-14)
        with torch.no_grad():
            scores = self.aesthetics_model(features)
            scores = scores[:, 0].detach().cpu().numpy()
        return scores
    
    def get_watermark_score(self, images):
        if isinstance(images, list):
            images = torch.stack([self.watermark_processor(image) for image in images]).cuda()
        elif isinstance(images, torch.Tensor):
            images = images.cuda()
        with torch.no_grad():
            pred = self.watermark_model(images)
            watermark_scores = F.softmax(pred, dim=1)[:,0].detach().cpu().numpy()

        return watermark_scores

    def predict_step(self, batch, batch_idx):
        images, texts = batch   
        # TODO 这里要么传入处理后的2种图片，要么传入纯图片，然后在下面的函数处理。（目前是传入纯图片）
        image_features = self.get_image_feature(images)
        text_features = self.get_text_feature(texts)
        clip_scores = self.calculate_clip_score(image_features, text_features)
        aes_scores = self.get_aesthetics_score(image_features)
        watermark_scores = self.get_watermark_score(images)
        return clip_scores, aes_scores, watermark_scores

    def on_predict_epoch_end(self, outputs):
        # 此处返回所有预测结果
        clip_scores = torch.cat([output[0] for output in outputs], dim=0)
        aes_scores = torch.cat([output[1] for output in outputs], dim=0)
        watermark_scores = torch.cat([output[2] for output in outputs], dim=0)
        return clip_scores, aes_scores, watermark_scores